In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from tensorflow.keras.models import load_model

In [15]:
# Load and Preprocess the Dataset
def dataset(defective_dir, non_defective_dir):
    images = []
    labels = []

    # Defective Dataset:
    for f_name in os.listdir(defective_dir):
        path = os.path.join(defective_dir, f_name)
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            image = cv2.resize(image, (128, 128))  # Resize for consistency
            images.append(image)
            labels.append(1)

    # Non-Defective Dataset:
    for f_name in os.listdir(non_defective_dir):
        path = os.path.join(non_defective_dir, f_name)
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            image = cv2.resize(image, (128, 128))  # Resize for consistency
            images.append(image)
            labels.append(0)

    return np.array(images), np.array(labels)

In [16]:
# Data Augmentation
def augment_data(images, labels):
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    augmented_images = []
    augmented_labels = []

    # The images are already (128, 128, 3), so no need to expand the last dimension again
    for image, label in zip(images, labels):
        image = np.expand_dims(image, axis=0)  # Add batch dimension to (1, 128, 128, 3)
        it = datagen.flow(image, batch_size=1)
        for _ in range(5):  # Generate 5 augmented versions of each image
            batch = next(it)
            augmented_images.append(batch[0].astype(np.uint8))  # Remove the batch dimension
            augmented_labels.append(label)

    return np.array(augmented_images), np.array(augmented_labels)

In [17]:
# Create and Train the CNN Model using Transfer Learning and Fine-tuning
def create_transfer_model(input_shape):
    base_model = VGG16(include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers[:-4]:  # Fine-tuning the last 4 layers
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [18]:
def main_train(defective_dir, non_defective_dir):
    # Load and preprocess dataset
    images, labels = dataset(defective_dir, non_defective_dir)

    # Normalize images and convert them to RGB format
    images = images / 255.0  # Normalize pixel values to [0, 1]
    images_rgb = np.repeat(images[..., np.newaxis], 3, axis=-1)  # Convert grayscale to RGB

    # Apply data augmentation
    augmented_images, augmented_labels = augment_data(images_rgb, labels)

    # Combine original and augmented data
    images_rgb = np.concatenate([images_rgb, augmented_images])
    labels = np.concatenate([labels, augmented_labels])

    # K-Fold cross-validation
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_no = 1
    for train, test in kfold.split(images_rgb, labels):
        print(f'Training fold {fold_no}...')
        model = create_transfer_model(images_rgb.shape[1:])

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10),
            ModelCheckpoint(f'model_fold_{fold_no}.keras', save_best_only=True)
        ]

        model.fit(images_rgb[train], labels[train], epochs=50, batch_size=16, validation_data=(images_rgb[test], labels[test]), callbacks=callbacks)

        loss, accuracy = model.evaluate(images_rgb[test], labels[test])
        print(f'Fold {fold_no} - Loss: {loss}, Accuracy: {accuracy}')
        fold_no += 1

    return model

In [19]:
# Function to identify cracks
def identify_crack(image_path, model_path):
    model = load_model(model_path)

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is not None:
        image = cv2.resize(image, (128, 128))  # Resize for consistency
        image = np.repeat(image[:, :, np.newaxis], 3, axis=2)  # Convert grayscale to RGB
        image = image / 255.0  # Normalize image
        image = np.expand_dims(image, axis=0)  # Add batch dimension

        prediction = model.predict(image)

        print("Prediction: ", prediction)
        return "Defective" if prediction >= 0.5 else "Non-Defective"
    else:
        return 'Image not loaded'

In [3]:
if __name__ == "__main__":
    # Directories for defective and non-defective images
    defective_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/train/Defective"
    non_defective_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/train/Non defective"

    model = main_train(defective_dir, non_defective_dir)
    model.save('crack_detection_model.keras')

Training fold 1...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 18s 114ms/step - accuracy: 0.4990 - loss: 0.7338 - val_accuracy: 0.5194 - val_loss: 0.6857
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.5476 - loss: 0.6806 - val_accuracy: 0.5361 - val_loss: 0.6759
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.5556 - loss: 0.6623 - val_accuracy: 0.5389 - val_loss: 0.6796
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 59ms/step - accuracy: 0.5691 - loss: 0.6338 - val_accuracy: 0.5389 - val_loss: 0.6994
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.5742 - loss: 0.6144 - val_accuracy: 0.5278 - val_loss: 0.7260
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.5820 - loss: 0.5915 - val_accuracy: 0.5417 - val_loss: 0.7738
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.6080 - loss: 0.5708 - val_accuracy: 0.5361 - val_loss: 0.7797
Epoch 8/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 

In [4]:
test_image_path = "/content/drive/MyDrive/Colab Notebooks/dataset/test/Non defective/IMG_20201114_101907.jpg" # Non-Defective
result = identify_crack(test_image_path, 'crack_detection_model.keras')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction:  [[0.00109123]]
Non-Defective


In [5]:
test_image_path = "/content/drive/MyDrive/Colab Notebooks/dataset/valid/Defective/IMG_20201114_102159.jpg" # Defective
result = identify_crack(test_image_path, 'crack_detection_model.keras')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
Prediction:  [[0.9996947]]
Defective


In [12]:
valid_image_path = "/content/drive/MyDrive/Colab Notebooks/dataset/valid/Defective/IMG_20201114_102417.jpg" # Defective
result = identify_crack(valid_image_path, 'crack_detection_model.keras')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
Prediction:  [[0.9445605]]
Defective


In [10]:
valid_image_path = "/content/drive/MyDrive/Colab Notebooks/dataset/valid/Non defective/IMG_20201114_100431.jpg" # Non-Defective
result = identify_crack(valid_image_path, 'crack_detection_model.keras')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
Prediction:  [[1.393427e-07]]
Non-Defective
